# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [57]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import re

#import nltk stuff
import nltk

nltk.download(['punkt','stopwords', 'wordnet', 'punkt_tab'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#import sklearn stuff
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\P334257\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM TAB_DR', engine)
X = df.message
y = df.drop(columns=['id', 'message', 'original', 'genre']) 

#check shapes
print(f"Shape of X: {X.shape} // Shape of Y: {y.shape}")
df.head()

Shape of X: (26216,) // Shape of Y: (26216, 36)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
#check data quality
condition = (y > 1) | (y < 0)
rows = condition.any(axis=1)
rows.sum()

np.int64(188)

So 188 rows contain data, that shouldn't be contained!

In [120]:
index = df[rows].index
index

Index([  117,   219,   305,   460,   576,   655,   882,   896,   924,   930,
       ...
       17401, 18251, 18514, 19714, 20051, 20324, 20495, 22326, 23382, 25215],
      dtype='int64', length=188)

### 2. Write a tokenization function to process your text data

In [59]:
def tokenize(text):
    #Normalize Text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    #Tokenize Text
    tokens = word_tokenize(text)

    #Remove Stopwords
    words = [t for t in tokens if t not in stopwords.words("english")]

    #Lemmatize
    clean_tokens = []
    for word in words:
        clean_tok = WordNetLemmatizer().lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\P334257\AppData\Local\Temp\1\ipykernel_23568\1877892729.py:3: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [60]:
#Test tokenize function
text = "The first time you see The Second Renaissance it may look boring. Look at it at least twice and definitely watch part 2. It will change your view of the matrix. Are the human people the ones who started the war ? Is AI a bad thing ?"
text = tokenize(text)
print(text)

['the', 'first', 'time', 'see', 'the', 'second', 'renaissance', 'may', 'look', 'boring', '.', 'look', 'least', 'twice', 'definitely', 'watch', 'part', '2', '.', 'it', 'change', 'view', 'matrix', '.', 'are', 'human', 'people', 'one', 'started', 'war', '?', 'is', 'ai', 'bad', 'thing', '?']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [68]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier(n_estimators=5))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [83]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.values)
print(X_test.values)
print(y_train)
print(y_test)



["RT ikePortnoyDT: Well,the Santiago show is on! However, its been moved to Friday..it's out of our hands, but at least the show will go on"
 'BON REPOS ROUTE MOLEAD number 2 inp(?) strong Union Committee request help. We have gotten no help. The big problem we try to locate(?) water, food. '
 "Even in the northern regional capital of Kidal, a stronghold of Mali's Tuareg rebellion and the scene of recent deadly ethnic violence, voters cast their ballots in an atmosphere of calm, although the turnout was thought to have been low."
 ...
 'Hello people in the courtyard of Miki at fontamara 27 Menos street extension, water please, do something'
 'Each day, 1,700 women go into labour and hundreds experience complications requiring life-saving medical intervention.'
 'This scenario could destabilize the entire Horn of Africa region, say the three aid agencies.']
['Please ,give me some help.Thanks before. '
 'A positive development is that cooling has been restored to the reactor pressure ves

In [72]:
#Train pipeline
pipeline.fit(X_train.values, y_train)

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000027251B6D440>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=5))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [75]:
y_pred = pipeline.predict(X_test)

In [80]:
print(f"y_pred Shape: {y_pred.shape} and data:")
print(y_pred)
print(f"y_test Shape: {y_test.shape} and data:")
print(y_test.values)
print("Columns:")
print(y.columns.values)

y_pred Shape: (6554, 36) and data:
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
y_test Shape: (6554, 36) and data:
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 1 1 0]
 ...
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
Columns:
['related' 'request' 'offer' 'aid_related' 'medical_help'
 'medical_products' 'search_and_rescue' 'security' 'military'
 'child_alone' 'water' 'food' 'shelter' 'clothing' 'money'
 'missing_people' 'refugees' 'death' 'other_aid' 'infrastructure_related'
 'transport' 'buildings' 'electricity' 'tools' 'hospitals' 'shops'
 'aid_centers' 'other_infrastructure' 'weather_related' 'floods' 'storm'
 'fire' 'earthquake' 'cold' 'other_weather' 'direct_report']


In [105]:
df_y_pred = pd.DataFrame(y_pred, columns=y.columns.values)
df_y_test = pd.DataFrame(y_test, columns=y.columns.values)
df_y_pred.value_counts()

related  request  offer  aid_related  medical_help  medical_products  search_and_rescue  security  military  child_alone  water  food  shelter  clothing  money  missing_people  refugees  death  other_aid  infrastructure_related  transport  buildings  electricity  tools  hospitals  shops  aid_centers  other_infrastructure  weather_related  floods  storm  fire  earthquake  cold  other_weather  direct_report
1        0        0      0            0             0                 0                  0         0         0            0      0     0        0         0      0               0         0      0          0                       0          0          0            0      0          0      0            0                     0                0       0      0     0           0     0              0                2318
0        0        0      0            0             0                 0                  0         0         0            0      0     0        0         0      0            

In [108]:
for col in df_y_pred.columns:
    print(df_y_pred[col].value_counts())

related
1    5300
0    1241
2      13
Name: count, dtype: int64
request
0    5814
1     740
Name: count, dtype: int64
offer
0    6554
Name: count, dtype: int64
aid_related
0    4568
1    1986
Name: count, dtype: int64
medical_help
0    6464
1      90
Name: count, dtype: int64
medical_products
0    6498
1      56
Name: count, dtype: int64
search_and_rescue
0    6537
1      17
Name: count, dtype: int64
security
0    6554
Name: count, dtype: int64
military
0    6525
1      29
Name: count, dtype: int64
child_alone
0    6554
Name: count, dtype: int64
water
0    6348
1     206
Name: count, dtype: int64
food
0    6129
1     425
Name: count, dtype: int64
shelter
0    6284
1     270
Name: count, dtype: int64
clothing
0    6539
1      15
Name: count, dtype: int64
money
0    6540
1      14
Name: count, dtype: int64
missing_people
0    6553
1       1
Name: count, dtype: int64
refugees
0    6534
1      20
Name: count, dtype: int64
death
0    6519
1      35
Name: count, dtype: int64
other_aid
0    6

In [104]:
#classification report on test data
for label in y.columns.values:
    print(label)
    print(classification_report(df_y_test[label], df_y_pred[label]))

related
              precision    recall  f1-score   support

           0       0.56      0.44      0.49      1563
           1       0.83      0.89      0.86      4944
           2       0.46      0.13      0.20        47

    accuracy                           0.78      6554
   macro avg       0.62      0.49      0.52      6554
weighted avg       0.76      0.78      0.77      6554

request
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      5443
           1       0.71      0.47      0.57      1111

    accuracy                           0.88      6554
   macro avg       0.81      0.72      0.75      6554
weighted avg       0.87      0.88      0.87      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      655

c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\P334257\Documents\Udacity\04_Data_Science\03_Data_Engineering\10_Project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.